Classifier

In [98]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm

In [4]:
# Load your own training data from a CSV file
train_data = pd.read_csv('faq_data.csv')

In [5]:
train_data.head()

,question,category
0,How do I locate my card?,11
1,"I still have not received my new card, I order...",11
2,I ordered a card but it has not arrived. Help ...,11
3,Is there a way to know when my card will arrive?,11
4,My card has not arrived yet.,11


In [6]:
# Define the FAQ categories
faq_categories = {
    0: 'card arrival',
    1: 'card linking',
    2: 'exchange rate',
    3: 'card payment wrong exchange rate',
    4: 'extra charge on statement',
    5: 'pending cash withdrawal',
    6: 'fiat currency support',
    7: 'card delivery estimate',
    8: 'automatic top up',
    9: 'card not working',
    10: 'exchange via app',
    11: 'lost or stolen card',
    12: 'age limit',
    13: 'pin blocked',
    14: 'contactless not working',
    15: 'top up by bank transfer charge',
    16: 'pending top up',
    17: 'cancel transfer',
    18: 'top up limits',
    19: 'wrong amount of cash received',
    20: 'card payment fee charged',
    21: 'transfer not received by recipient',
    22: 'supported cards and currencies',
    23: 'getting virtual card',
    24: 'card acceptance',
    25: 'top up reverted',
    26: 'balance not updated after cheque or cash deposit',
    27: 'card payment not recognised',
    28: 'edit personal details',
    29: 'why verify identity',
    30: 'unable to verify identity',
    31: 'get physical card',
    32: 'visa or mastercard',
    33: 'topping up by card',
    34: 'disposable card limits',
    35: 'compromised card',
    36: 'atm support',
    37: 'direct debit payment not recognised',
    38: 'passcode forgotten',
    39: 'declined cash withdrawal',
    40: 'pending card payment',
    41: 'lost or stolen phone',
    42: 'request refund',
    43: 'declined transfer',
    44: 'Refund not showing up',
    45: 'declined card payment',
    46: 'pending transfer',
    47: 'terminate account',
    48: 'card swallowed',
    49: 'transaction charged twice',
    50: 'verify source of funds',
    51: 'transfer timing',
    52: 'reverted card payment?',
    53: 'change pin',
    54: 'beneficiary not allowed',
    55: 'transfer fee charged',
    56: 'receiving money',
    57: 'failed transfer',
    58: 'transfer into account',
    59: 'verify top up',
    60: 'getting spare card',
    61: 'top up by cash or cheque',
    62: 'order physical card',
    63: 'virtual card not working',
    64: 'wrong exchange rate for cash withdrawal',
    65: 'get disposable virtual card',
    66: 'top up failed',
    67: 'balance not updated after bank transfer',
    68: 'cash withdrawal not recognised',
    69: 'exchange charge',
    70: 'top up by card charge',
    71: 'activate my card',
    72: 'cash withdrawal charge',
    73: 'card about to expire',
    74: 'apple pay or google pay',
    75: 'verify my identity',
    76: 'country support',
}

In [7]:
# Load a pre-trained spaCy model for entity recognition
nlp = spacy.load('en_core_web_sm')

In [8]:
# Load the pre-trained language model
language_model = "gpt2"  # or other language models like "gpt2-medium", "gpt2-large"

In [9]:
# Function to extract entities using spaCy
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

In [13]:
# Train the classifier
def train_classifier():
    vectorizer = TfidfVectorizer()
    
    # Load your own training data from a CSV file
    train_data = pd.read_csv('faq_data.csv')
    X_train = vectorizer.fit_transform(train_data['question'])
    y_train = train_data['category']
    
    # Train a LinearSVC classifier
    classifier = LinearSVC() #classifier = SVC()
    classifier.fit(X_train, y_train)
    
    return vectorizer, classifier

In [ ]:
# Perform grid search with cross-validation
grid_search = GridSearchCV(classifier, param_grid, cv=5)
grid_search.fit(X_train_vectors, y_train)

In [ ]:
# Get the best hyperparameters and classifier
best_params = grid_search.best_params_
best_classifier = grid_search.best_estimator_

In [ ]:
# Make predictions on the test set using the best classifier
predictions = best_classifier.predict(X_test_vectors)

# Evaluate the classifier
accuracy = accuracy_score(y_test, predictions)
recall = recall_score(y_test, predictions, average='weighted')

print("Best Hyperparameters:", best_params)
print("Accuracy:", accuracy)
print("Recall:", recall)

In [14]:
# Classify customer questions using the trained model
def classify_question(question, vectorizer, classifier):
    X = vectorizer.transform([question])
    
    # Classify the customer question
    category_id = classifier.predict(X)[0]
    category = faq_categories[category_id]
    
    return category

In [15]:
# Test the classifier and entity recognition
vectorizer, classifier = train_classifier()

In [88]:
print("Chatbot: Hello. How can I help you?\n***Enter bye/end/quit to end the chat***") 
while True:
    
    # Get user input
    user_input = input("User: ")
    
    # Check if user wants to end the conversation
    if user_input.lower() in ["bye", "end", "quit"]:
        print("ChatBot: Goodbye!")
        break

    # Remove the "User:" prefix from the input
    user_input = user_input.replace("User: ", "").strip()
    
    category = classify_question(user_input, vectorizer, classifier)
    entities = extract_entities(user_input)
    
    print("Category:", category)
    print("Entities:", entities)

Chatbot: Hello. How can I help you?
***Enter bye/end/quit to end the chat***


User:  what


ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

GPT2

In [37]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config

In [18]:
# Download and load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

C:\Users\sahap\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sahap\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [19]:
# Download and load the GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [32]:
# Set the model to evaluation mode
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [54]:
# Customize the generation parameters
model_config = GPT2Config.from_pretrained("gpt2", config=model_config)
model_config.pad_token_id = model_config.eos_token_id

# Encode input text
input_text = "Hey how are you? i need some help changing the password to my bank account"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

In [55]:
# Generate output using the model
output = model.generate(
    input_ids,
    max_length=100,
    #config=model_config,
    num_return_sequences=1,  # Number of different sequences to generate
    do_sample=True,  # Enable random sampling
    temperature=0.7,  # Controls randomness, higher value -> more random
    repetition_penalty=1.0,  # Controls repetition, higher value -> less repetition
    length_penalty=1.0,  # Controls output length, lower value -> shorter length
)

In [56]:
# Decode and print the generated response
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(output_text)

Hey how are you? i need some help changing the password to my bank account now. i need this to be saved. so i can pay for this. i need to get my car back to the dealership. i need the car keys to the dealership.Give me this and i'll pay for the car keys.I can use this to get my car back to the dealership.I need to get my car back to the dealership.

Anonymous 02/22/16 (Thu) 07


Save GP2 and run locally

In [70]:
import torch

In [65]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Download and load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Download and load the GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Download and load the GPT-2 model
model_config = GPT2Config.from_pretrained("gpt2")

# Save the tokenizer and model to your local computer
tokenizer.save_pretrained("C:/Users/sahap/Documents/Chatbot/tokenizer")
model.save_pretrained("C:/Users/sahap/Documents/Chatbot/model")
model_config.save_pretrained("C:/Users/sahap/Documents/Chatbot/model_config")

In [67]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the tokenizer and model from the local saved files
tokenizer = GPT2Tokenizer.from_pretrained("C:/Users/sahap/Documents/Chatbot/tokenizer")
model_config = GPT2Config.from_pretrained("C:/Users/sahap/Documents/Chatbot/model_config")
model = GPT2LMHeadModel.from_pretrained("C:/Users/sahap/Documents/Chatbot/model", config=model_config)

In [68]:
# Set the attention mask and pad token ID
model.config.pad_token_id = model.config.eos_token_id

In [84]:
while True:
    # Get user input
    user_input = input("User: ")
    
    # Check if user wants to end the conversation
    if user_input.lower() in ["bye", "end", "quit"]:
        print("ChatBot: Goodbye!")
        break

    # Remove the "User:" prefix from the input
    user_input = user_input.replace("User: ", "").strip()
    
    # Tokenize the user input
    input_ids = tokenizer.encode(""+user_input, return_tensors="pt")

    # Set the attention mask
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

    # Generate response using the model
    output = model.generate(
        input_ids,
        max_length=100,
        num_return_sequences=1,  # Number of different sequences to generate
        do_sample=True,  # Enable random sampling
        temperature=0.2,  # Controls randomness, higher value -> more random 0.7
        repetition_penalty=7.0,  # Controls repetition, higher value -> less repetition
        length_penalty=0.5,  # Controls output length, lower value -> shorter length
    )

    # Convert the output ids to text
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Remove the input text from the generated response
    response = response.replace(user_input, "").strip()

    print("ChatBot: ", response)

User:  i want to open a new account but I am unable to contact my account banker.


ChatBot:  summarize: 
I have been told that the bank will not be able, and they are going through with it because there is no way of getting an answer from them about what happened in this case (i think). So if you would like me on your side please send us some information so we can find out more details as soon possible!


User:  bye


ChatBot: Goodbye!


In [59]:
# Set the model to evaluation mode
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [61]:
# Set the maximum length for generated responses
max_response_length = 100

In [63]:
#Customize the generation parameters
model_config = GPT2Config.from_pretrained("gpt2", config=model_config)
model_config.pad_token_id = model_config.eos_token_id


In [64]:
while True:
    # Get user input
    user_input = input("User: ")

    # Tokenize the user input
    input_ids = tokenizer.encode(user_input, return_tensors="pt")

    # Generate response using the model
    output = model.generate(
        input_ids,
        max_length=max_response_length,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.7,
        repetition_penalty=1.0,
        length_penalty=1.0,
    )

    # Convert the output ids to text
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    print("ChatBot:", response)

User:  hello how are you today?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ChatBot: hello how are you today?

I'm sorry, but my first love was to my sister.

What do you do for a living?

I live as a student.

My parents died.

Did you go to university?

Yes.

You have an amazing job?rafted in?

Yes.

You don't work at a big corporate organization?

No, I don't work at a big corporate organization.




KeyboardInterrupt: Interrupted by user

In [85]:
from transformers import pipeline

In [86]:
# Allocate a pipeline for sentiment-analysis
classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'POSITIVE', 'score': 0.9996980428695679}]

In [87]:
classifier('This is ridiculous. This is the fourth time of me trying to call')

[{'label': 'NEGATIVE', 'score': 0.9994600415229797}]

In [ ]:
###Save DistillBert and run locally

In [89]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

In [91]:
model_name = 'distilbert-base-uncased'  # or other DistilBERT variants
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
classification_model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=77)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier.weight', 'pre_classifier.

In [92]:
# Save the tokenizer and model to your local computer
tokenizer.save_pretrained("C:/Users/sahap/Documents/Chatbot/DistilBert/tokenizer")
classification_model.save_pretrained("C:/Users/sahap/Documents/Chatbot/DistilBert/model")

In [97]:
# Load the tokenizer and model from the local saved files
tokenizer = DistilBertTokenizer.from_pretrained("C:/Users/sahap/Documents/Chatbot/DistilBert/tokenizer")
classification_model = DistilBertForSequenceClassification.from_pretrained("C:/Users/sahap/Documents/Chatbot/DistilBert/model", num_labels=77)

In [99]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score

# Load the dataset
data = pd.read_csv('faq_data.csv')

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['question'], data['category'], test_size=0.2, random_state=42)

# Perform feature extraction using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)

# Train the SVM classifier
classifier = LinearSVC()
classifier.fit(X_train_vectors, y_train)

# Make predictions on the test set
predictions = classifier.predict(X_test_vectors)

# Evaluate the classifier
accuracy = accuracy_score(y_test, predictions)
recall = recall_score(y_test, predictions, average='weighted')

print("Accuracy:", accuracy)
print("Recall:", recall)


Accuracy: 0.8972105464272068
Recall: 0.8972105464272068


In [100]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC

# Create a parameter grid for hyperparameter tuning
param_grid = {'C': [0.1, 1, 10]}

# Create a LinearSVC classifier
classifier = LinearSVC()

# Perform grid search with cross-validation
grid_search = GridSearchCV(classifier, param_grid, cv=5)
grid_search.fit(X_train_vectors, y_train)

# Get the best hyperparameters and classifier
best_params = grid_search.best_params_
best_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best classifier
predictions = best_classifier.predict(X_test_vectors)

# Evaluate the classifier
accuracy = accuracy_score(y_test, predictions)
recall = recall_score(y_test, predictions, average='weighted')

print("Best Hyperparameters:", best_params)
print("Accuracy:", accuracy)
print("Recall:", recall)

Best Hyperparameters: {'C': 1}
Accuracy: 0.8972105464272068
Recall: 0.8972105464272068


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

# Load the dataset
data = pd.read_csv('faq_data.csv')

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['question'], data['category'], test_size=0.2, random_state=42)

# Perform feature extraction using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)

# Define the parameter grid for hyperparameter tuning
param_grid = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10]}

# Create an SVM classifier
classifier = SVC()

# Perform grid search with cross-validation
grid_search = GridSearchCV(classifier, param_grid, cv=5)
grid_search.fit(X_train_vectors, y_train)

# Get the best hyperparameters and classifier
best_params = grid_search.best_params_
best_classifier = grid_search.best_estimator_

# Make predictions on user input
while True:
    user_input = input("Enter your question (or 'q' to quit): ")
    if user_input == 'q':
        break

    # Transform user input into vector using the same vectorizer
    user_input_vector = vectorizer.transform([user_input])

    # Predict the category of the user input
    prediction = best_classifier.predict(user_input_vector)[0]

    # Frame the category as a sentence
    category_sentence = f"The category of your question is: {prediction}"

    print(category_sentence)

################# FINAl CATEGORY AND NER ################

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

# Load the dataset
data = pd.read_csv('faq_data.csv')

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['question'], data['category'], test_size=0.2, random_state=42)

# Perform feature extraction using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)

# Define the parameter grid for hyperparameter tuning
param_grid = {'C': [0.1, 1, 10]}

# Create a LinearSVC classifier
classifier = LinearSVC()

# Perform grid search with cross-validation
grid_search = GridSearchCV(classifier, param_grid, cv=5)
grid_search.fit(X_train_vectors, y_train)

# Get the best hyperparameters and classifier
best_params = grid_search.best_params_
best_classifier = grid_search.best_estimator_

# Load Stanford NER model
stanford_ner_model = 'path_to_stanford_ner_model'
stanford_ner_jar = 'path_to_stanford_ner_jar'

# Create NER tagger
ner_tagger = StanfordNERTagger(stanford_ner_model, stanford_ner_jar, encoding='utf-8')

# Function to extract entities from the question using NER
def extract_entities(question):
    tokenized_question = word_tokenize(question)
    entities = ner_tagger.tag(tokenized_question)
    return [entity for entity, tag in entities if tag != 'O']

# Function to classify user input and frame the response
def classify_question(user_input):
    user_input_vector = vectorizer.transform([user_input])
    predicted_category = best_classifier.predict(user_input_vector)[0]
    entities = extract_entities(user_input)
    response = f"The category of your question is {predicted_category}. The entities mentioned are: {', '.join(entities)}."
    return response

# Chatbot loop
while True:
    user_input = input("Please enter your question (or 'q' to quit): ")
    if user_input.lower() == 'q':
        break
    else:
        response = classify_question(user_input)
        print(response)
